In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Fork-Join network example 7
val model = Network("model")

In [ ]:
// Create network
val delay = Delay(model, "Delay")
val fork1 = Fork(model, "Fork1")
val join1 = Join(model, "Join1", fork1)
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)

val jobclass1 = ClosedClass(model, "class1", 1, delay, 0)
val jobclass2 = ClosedClass(model, "class2", 1, delay, 0)

In [ ]:
// Service configurations
delay.setService(jobclass1, Exp(0.25))
queue1.setService(jobclass1, Exp(2.0))
queue2.setService(jobclass1, Exp(2.0))

delay.setService(jobclass2, Exp(0.25))
queue1.setService(jobclass2, Exp(2.0))
queue2.setService(jobclass2, Exp(2.0))

In [ ]:
// Set routing matrix with class switching
val P = model.initRoutingMatrix()

// Class 1 routing
P.set(jobclass1, jobclass1, delay, fork1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join1, 1.0)
P.set(jobclass1, jobclass1, queue2, join1, 1.0)
P.set(jobclass1, jobclass1, join1, delay, 1.0)

// Class 2 routing (note the class switching from jobclass2 to jobclass1)
P.set(jobclass2, jobclass2, delay, fork1, 1.0)
P.set(jobclass2, jobclass1, fork1, queue1, 1.0)
P.set(jobclass2, jobclass1, fork1, queue2, 1.0)

model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(MVA(model))
solvers.add(JMT(model, "seed", 23000))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name.replace("Solver", "")}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This post-fork class switching model demonstrates:

1. **Post-Fork Class Switching**:
   - **Class 1**: Complete cycle: Delay → Fork → {Queue1, Queue2} → Join → Delay
   - **Class 2**: Partial cycle: Delay → Fork → switches to Class 1 in queues
   - Class switching occurs immediately after fork operation

2. **Asymmetric Class Behavior**:
   - **Class 1 jobs**: Follow normal fork-join semantics
   - **Class 2 jobs**: Transform into Class 1 jobs after forking
   - Results in effective job injection into Class 1 population

3. **Closed System with Mixed Populations**:
   - 1 job of each class initially
   - Class 2 jobs convert to Class 1 jobs
   - Dynamic population redistribution

4. **Service Configuration**:
   - Identical service rates for both classes (rate 2.0)
   - Delay station rate 0.25 (bottleneck)
   - PS scheduling provides fair resource sharing

5. **Performance Implications**:
   - **Population concentration**: Class 1 effectively receives more jobs
   - **Unbalanced utilization**: Only Class 1 statistics matter long-term
   - **Fork amplification**: Class 2 jobs create additional Class 1 load

6. **Synchronization Effects**:
   - Only Class 1 jobs participate in join synchronization
   - Class 2 jobs disappear after fork (become Class 1)
   - Join waits only for Class 1 parallel tasks

The model represents:
- Job promotion or escalation systems
- Task refinement processes where initial jobs become detailed work
- Systems with job transformation after parallel dispatch
- Manufacturing with part classification after inspection

Key insight: Post-fork class switching creates effective job multiplication, where one class feeds into another after parallel processing begins, leading to dynamic load rebalancing.

**Note**: This pattern can lead to solver differences as analytical and simulation methods may handle class switching transitions differently.